# Amazon SageMaker Tensorflow Containerを使用したハイパーパラメータチューニング

このチュートリアルでは、**SageMaker TensorFlowコンテナ** を使用して[MNISTデータセット](http://yann.lecun.com/exdb/mnist/) をトレーニングするための畳み込みニューラルネットワークモデルを作成する方法に焦点を当てています。 ハイパーパラメータチューニングを活用して、さまざまなハイパーパラメータの組み合わせを使用して複数のトレーニングジョブを開始し、最良のモデルトレーニング結果を持つトレーニングを見つけます。

## 環境を設定する
ワークフローを開始する前に環境を設定します。

1. トレーニングデータセットとモデル成果物が格納されるS3バケットとプレフィックスを指定する。
1. SageMakerに渡される実行ロールを取得して、S3バケットなどのリソースにアクセスします。

In [2]:
import sagemaker

bucket = sagemaker.Session().default_bucket() # we are using a default bucket here but you can change it to any bucket in your account
prefix = 'sagemaker/DEMO-hpo-tensorflow-high' # you can customize the prefix (subfolder) here

role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-314676777416


次に、必要なPythonライブラリをインポートします。

In [3]:
import boto3
from time import gmtime, strftime
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

## MNISTデータセットのダウンロード

In [4]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Instructions for updating:
Please use tf.data to implement this functionality.


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz


Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
('Writing', 'data/train.tfrecords')
('Writing', 'data/validation.tfrecords')
('Writing', 'data/test.tfrecords')


## データをアップロードする
私たちは ```sagemaker.Session.upload_data``` 関数を使ってデータセットをS3の場所にアップロードします。 戻り値の`input`は、トレーニングジョブの開始時に使用します。

In [5]:
inputs = sagemaker.Session().upload_data(path='data', bucket=bucket, key_prefix=prefix+'/data/mnist')
print (inputs)

s3://sagemaker-us-east-1-314676777416/sagemaker/DEMO-hpo-tensorflow-high/data/mnist


## 分散学習用のスクリプトを作成する
ネットワークモデルの完全なコードは次のとおりです。

In [7]:
!pygmentize 'mnist.py'

import os
import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

INPUT_TENSOR_NAME = 'inputs'
SIGNATURE_NAME = 'predictions'

def model_fn(features, labels, mode, params):
    # this script takes learning_rate as a hyperparameter
    learning_rate = params.get("learning_rate",0.1)
    
    # Input Layer
    input_layer = tf.reshape(features[INPUT_TENSOR_NAME], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

   


このスクリプトは、[TensorFlow MNIST example](https://github.com/tensorflow/models/tree/master/official/mnist) の拡張です。これは、`model_fn(features, labels, mode)` を提供し、学習、評価、推論に使用されます。

### 通常の ```model_fn```

通常の**``` model_fn```**は、下記のような流れになります。
1. [ニューラルネットワークを定義する](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L96)
 -  [ニューラルネットワークで ```features```を適用する。](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L178)
 -  [``mode``が ``PREDICT``であれば、ニューラルネットワークの出力を返す。](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py＃L186)
 -  [出力を ``labels``と比較する損失関数を計算する](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L188)
 -  [オプティマイザを作成し、損失関数を最小化し、ニューラルネットワークを改善する。](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L193)
 -  [出力、オプティマイザ、損失関数を返す。](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L205)

### 分散学習のための ```model_fn```を書く
分散トレーニングが行われると、複数のトレーニングインスタンスに同じニューラルネットワークが送信されます。各インスタンスは、データセットのバッチを予測し、損失を計算し、オプティマイザを最小化します。このプロセスのループ全体を **トレーニングステップ** と呼びます。

### 同期トレーニングのステップ
[グローバルステップ](https://www.tensorflow.org/api_docs/python/tf/train/global_step)は、インスタンス間で共有されるグローバル変数です。分散学習では、オプティマイザは実行間の **トレーニングステップ** の数を追跡する必要があります。

```python
train_op = optimizer.minimize（loss、tf.train.get_or_create_global_step（））
```
分散学習のために必要な変更は、たったこれだけです！

## ハイパーパラメータチューニングジョブを設定する
*以下のデフォルト設定では、ハイパーパラメータチューニングジョブは完了するまでに約30分かかります。

ここで、SageMaker Python SDKを使用して、以下の手順に従って、ハイパーパラメータチューニングジョブを設定します。
* TensorFlowトレーニングジョブを設定する``estimator``を作成します。
* チューニングする予定のハイパーパラメータの範囲を定義します。この例では、 ```learning_rate```を調整しています。
* 最適化するチューニングジョブの客観的なメトリックを定義します。
* 上記の設定でハイパーパラメータチューナーを作成し、リソース設定を調整します。
* ```batch_size```など、別のハイパーパラメータも同時に最適化にかけられます。

SageMakerで単一のTensorFlowジョブを習得するのと同様に、TensorFlowスクリプト、IAMロール、および（ジョブごとの）インスタンス構成を渡すTensorFlow estimatorを定義します。

In [18]:
estimator = TensorFlow(entry_point='mnist.py',
                  role=role,
                  framework_version='1.11.0',
                  training_steps=1000, 
                  evaluation_steps=100,
                  train_instance_count=4,
#                  train_instance_type='ml.m4.xlarge',
                  train_instance_type='ml.c5.2xlarge',
                  base_job_name='DEMO-hpo-tensorflow')

`Estimator`を定義したら、調整したいハイパーパラメーターとその可能な値を指定できます。 我々は、3つの異なるタイプのハイパーパラメータを有する。
 - カテゴリカルパラメータは、離散集合から1つの値を取る必要があります。 これを定義するには、可能な値のリストを `CategoricalParameter（list）`に渡します。
 - 連続パラメータは、 `ContinuousParameter（min、max）`で定義される最小値と最大値の間の任意の実数値をとることができます。
 - 整数パラメータは、IntegerParameter（min、max）で定義される最小値と最大値の間の任意の整数値をとることができます。

*可能であれば、最も制限の少ないタイプとして値を指定することがほとんど常にベストであることに注意してください。 たとえば、学習率を0.01と0.2の間の連続値として調整すると、0.01、0.1、0.15、または0.2の値を持つカテゴリパラメータとしてチューニングするよりも良い結果が得られる可能性があります。

In [26]:
hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.2)}
#hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.2), 'batch_size':CategoricalParameter([50, 100, 200])}

Estimatorを定義したら、調整したいハイパーパラメーターとその可能な値を指定できます。ここでは、3つの異なるタイプのハイパーパラメーターを定義できます。
  * カテゴリカルパラメータは、離散集合から1つの値を取る必要があります。これを定義するには、可能な値のリストを `CategoricalParameter（list）`に渡します。
  * 連続パラメタは、 `ContinuousParameter（min、max）`で定義される最小値と最大値の間の任意の実数値をとることができます。
  * 整数パラメータは、`IntegerParameter（min、max）`で定義される最小値と最大値の間の任意の値をとることができます。

*可能であれば、できるだけ限られた適切な範囲で値を指定することで、より良い解が得られること注意してください。学習率を0.01と0.2の間の連続値として調整すると、0.01,0.1 、0.15、または0.2の値を持つカテゴリパラメタとしてチューニングするより良い結果が得られる可能性があります。

In [20]:
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss = ([0-9\\.]+)'}]

Now, we'll create a `HyperparameterTuner` object, to which we pass:
- The TensorFlow estimator we created above
- Our hyperparameter ranges
- Objective metric name and definition
- Tuning resource configurations such as Number of training jobs to run in total and how many training jobs can be run in parallel.

次に、`HyperparameterTuner`オブジェクトを作成します。
* 上記で作成したTensorFlow estimator
* ハイパーパラメータの範囲
* ターゲットメトリクス定義
* 合計で実行するトレーニング・ジョブの数、並行して実行できるトレーニング・ジョブの数などのリソース構成をチューニングします。

In [21]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
        　                    hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3,
                            objective_type=objective_type)

## ハイパーパラメータチューニングジョブを起動する
最後に、 `.fit（）`を呼び出してS3パスをtrainとtest datasetに渡すことで、ハイパープレーターチューニングジョブを開始することができます。

ハイパーパラメータチューニングジョブが作成されたら、次のステップでチューニングジョブの進捗状況を表示できるようになり、SageMakerのコンソール - >ジョブに移動して、ハイパーパラメータチューニングジョブの進行状況を確認できます。

In [22]:
tuner.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-314676777416
INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-tensorflow-181217-1814


ハイパーパラメータ調整ジョブの状態を素早くチェックして、正常に起動したことを確認しましょう。

In [27]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

u'Completed'

## チューニングジョブの結果を分析する - チューニングジョブが完了した後
チューニングジョブの結果を分析するためのサンプルコードについては、```HPO_Analyze_TuningJob_Results.ipynb```を参照してください。

## ベストモデルを展開する
最良のモデルを得たので、これをエンドポイントに導入することができます。 モデルを展開する方法については、他のSageMakerサンプルノートブックまたはSageMakerのマニュアルを参照してください。